# Subspace Solver

## Introduction

Mizore supports Quantum Subspace Diagonalization (QSD), as in the paper "[A non-orthogonal variational quantum eigensolver](https://iopscience.iop.org/article/10.1088/1367-2630/ab867b)".
$\newcommand\ket[1]{|#1\rangle} \newcommand\bra[1]{\langle#1|}$

This methods takes a set of quantum states $\Psi=\{\ket{\psi_i}\}$ that can be produced by known quantum circuit and 
diagonalize the Hamiltonian in the space spanned by the set $\Psi$.

The core procedure is to solve the generalized eigenvalue problem
$H\vec{c}=S\vec{c}E$, where $H_{ij}=\bra{\psi_i}H\ket{\psi_j}$, $S_{ij}=\bra{\psi_i}\psi_j\rangle$, $E$ is the eigenvalue and $\vec{c}$ is the eigenvector. The terms can be evaluated efficiently by quantum computers.



## Basic Usage

In Mizore, a QSD algorithm can be easily carried out as follows.

Here, we construct the subspace by adding all combinations of qubit excitations (X gates) to a subset of qubits (0th and 2ed). QSD in this subspace is equivalent to carrying out exact diagonalization in the subspace.

In [1]:
from HamiltonianGenerator import make_example_H2
from Blocks import BlockCircuit
from SubspaceSolver import add_local_complete_basis,SubspaceSolver
energy_obj=make_example_H2()
init_bc=BlockCircuit(4,init_block=energy_obj.init_block)
circuits=add_local_complete_basis([init_bc],(0,2))
for circuit in circuits:
    print(circuit)
qse_solver=SubspaceSolver(circuits,energy_obj.hamiltonian,progress_bar=False)
qse_solver.execute()

Symmetry: Dooh  is used when build the molecule.
Block Num:1; Qubit Num:4
Block list:
Type:HartreeFockInitBlock; Para Num:0; Qsubset:[0]
Block Num:2; Qubit Num:4
Block list:
Type:HartreeFockInitBlock; Para Num:0; Qsubset:[0]
Type:HartreeFockInitBlock; Para Num:0; Qsubset:(0,)
Block Num:2; Qubit Num:4
Block list:
Type:HartreeFockInitBlock; Para Num:0; Qsubset:[0]
Type:HartreeFockInitBlock; Para Num:0; Qsubset:(2,)
Block Num:2; Qubit Num:4
Block list:
Type:HartreeFockInitBlock; Para Num:0; Qsubset:[0]
Type:HartreeFockInitBlock; Para Num:0; Qsubset:(0, 2)
4 states used to construct the subspace, where the complete space is 16-dimensional.
Subspace Expansion Method Started
Calculating S matrix
[1. 1. 1. 1.]
Calculating H matrix
The ground state energy is -1.1372838344885023
It's eigenvector is [-0.99364675+0.j  0.        +0.j  0.        +0.j  0.11254389+0.j]


Because the complete basis of a qubit subset contains exponentially many state, we cannot use the above method on large qubit subset. A more practical subspace is spanned by the states generated in the *Krylov* way. Suppose we have a state $\ket{\psi}$ near the ground state, such as the Hartree-Fock state. The Krylov subspace is like $\operatorname{span}(\{e^{iHn\Delta t}\ket{\psi}|n=0,1,\cdots\})$. In Mizore, a QSD algorithm with Krylov subspace can be easily carried out as follows.

In [2]:
from SubspaceSolver import generate_krylov_circuits
delta_t=0.01
n_circuit=3
krylov_circuits=generate_krylov_circuits(init_bc,energy_obj.hamiltonian,delta_t,n_circuit)
for circuit in krylov_circuits:
    print(circuit)
qse_solver=SubspaceSolver(krylov_circuits,energy_obj.hamiltonian,progress_bar=False)
qse_solver.execute()

Block Num:2; Qubit Num:4
Block list:
Type:HartreeFockInitBlock; Para Num:0; Qsubset:[0]
Type:TimeEvolutionBlock; Para Num:1; TimeEvolution: T=0.01
Block Num:2; Qubit Num:4
Block list:
Type:HartreeFockInitBlock; Para Num:0; Qsubset:[0]
Type:TimeEvolutionBlock; Para Num:1; TimeEvolution: T=0.02
Block Num:2; Qubit Num:4
Block list:
Type:HartreeFockInitBlock; Para Num:0; Qsubset:[0]
Type:TimeEvolutionBlock; Para Num:1; TimeEvolution: T=0.03
3 states used to construct the subspace, where the complete space is 16-dimensional.
Subspace Expansion Method Started
Calculating S matrix
[1.41278372e-16 6.56702958e-06 2.99999343e+00]
Calculating H matrix
The ground state energy is -1.137283834269396
It's eigenvector is [-0.4203907 +3.10649666e+01j -0.16551246-2.91835191e-02j
  0.28585104-3.10391949e+01j]


In addition, one can build a parallel `SubspaceSolver` by assign a `TaskManager` to it.

In [3]:
from ParallelTaskRunner import TaskManager
task_manager=TaskManager(n_processor=6,task_package_size=10)
qse_solver=SubspaceSolver(krylov_circuits,energy_obj.hamiltonian,progress_bar=False)
qse_solver.execute()

3 states used to construct the subspace, where the complete space is 16-dimensional.
Subspace Expansion Method Started
Calculating S matrix
[1.41278372e-16 6.56702958e-06 2.99999343e+00]
Calculating H matrix
The ground state energy is -1.137283834269396
It's eigenvector is [-0.4203907 +3.10649666e+01j -0.16551246-2.91835191e-02j
  0.28585104-3.10391949e+01j]


## Sparse circuit 

### Motivation

The developers of Mizore believe that in near-term, practical chemical calculation by quantum computer will not be realized by having the quantum computer produce exactly the ground state. Rather, the quantum computer should produce the basis of a subspace that contains the ground state. The dimension of the subspace should be a intermediate number so that the states in the basis can be prepared by near-term devices and in the meantime give speed-up with respect to the classical methods like exact diagonalization (ED). 

We find that in previous efforts, people use either states that can be easily produced by very many (such as the computational basis which can be produced classically)(such as in ED), or state that is very difficult to produce (such as the true ground state)(such as in VQE). We conjecture that we can use *intermediate* number of partly entangled circuits, which is of *intermediate* difficulty to produce, to span the subspace that covers the ground state. The qubits in those circuits can be divided into separable sets. We call the circuits with many separable sets *sparse circuits*.

(See also the README file in the root path)

### Implementation

Because of the importance of sparse circuit mentioned above, Mizore provides intrinsic support for sparse circuit in `BlockCircuit` and `Blocks._sparse_utilities` as follows.

In [4]:
from Blocks import BlockCircuit,HardwareEfficientEntangler,PauliGatesBlock
bc=BlockCircuit(10)
print("Sparable qubits sets (Before entangling any qubits)")
print(bc.get_disjoint_active_sets())
bc.add_block(HardwareEfficientEntangler([1,2]))
print("Sparable qubits sets (After entangling 1,2)")
print(bc.get_disjoint_active_sets())
bc.add_block(HardwareEfficientEntangler([1,3]))
bc.add_block(HardwareEfficientEntangler([6,7]))
print("Sparable qubits sets (After entangling 1,3;6,7)")
print(bc.get_disjoint_active_sets())
bc.add_block(PauliGatesBlock([(1,'X'),(4,'Y'),(6,'Z'),(9,'X')]))
print("Sparable qubits sets (After adding a non-entangling block; Should remain)")
print(bc.get_disjoint_active_sets())

Sparable qubits sets (Before entangling any qubits)
[{0}, {1}, {2}, {3}, {4}, {5}, {6}, {7}, {8}, {9}]
Sparable qubits sets (After entangling 1,2)
[{0}, {1, 2}, {3}, {4}, {5}, {6}, {7}, {8}, {9}]
Sparable qubits sets (After entangling 1,3;6,7)
[{0}, {1, 2, 3}, {4}, {5}, {6, 7}, {8}, {9}]
Sparable qubits sets (After adding a non-entangling block; Should remain)
[{0}, {1, 2, 3}, {4}, {5}, {6, 7}, {8}, {9}]


In [5]:
from Blocks._sparse_circuit_utilities import get_localized_circuit_list
local_bc_list=list(get_localized_circuit_list(bc))
for local_bc in local_bc_list:
    print(local_bc)

Block Num:1; Qubit Num:10
Block list:
Type:PauliGatesBlock; Para Num:0; PauliString:[]
Block Num:3; Qubit Num:10
Block list:
Type:HardwareEfficientEntangler; Para Num:6; Qsubset:[1, 2]
Type:HardwareEfficientEntangler; Para Num:6; Qsubset:[1, 3]
Type:PauliGatesBlock; Para Num:0; PauliString:[(1, 'X')]
Block Num:1; Qubit Num:10
Block list:
Type:PauliGatesBlock; Para Num:0; PauliString:[(4, 'Y')]
Block Num:1; Qubit Num:10
Block list:
Type:PauliGatesBlock; Para Num:0; PauliString:[]
Block Num:2; Qubit Num:10
Block list:
Type:HardwareEfficientEntangler; Para Num:6; Qsubset:[6, 7]
Type:PauliGatesBlock; Para Num:0; PauliString:[(6, 'Z')]
Block Num:1; Qubit Num:10
Block list:
Type:PauliGatesBlock; Para Num:0; PauliString:[]
Block Num:1; Qubit Num:10
Block list:
Type:PauliGatesBlock; Para Num:0; PauliString:[(9, 'X')]


In [6]:
local_bc=local_bc_list[1]
pcircuit=local_bc.get_ansatz_on_active_position()
print("Number of qubits in reduced circuit:",pcircuit.n_qubit)

Number of qubits in reduced circuit: 3


Using sparse circuit can significantly accelarate the circuit evalution. Here, we provide a code to demonstrate this.

In [7]:
from Blocks import BlockCircuit,HardwareEfficientEntangler
from Blocks._sparse_circuit_utilities import get_0000_amplitude_on_sparse_circuit
from Blocks._utilities import get_0000_amplitude_on_circuit
from time import time
n_qubit=24 # Adjustable!!
bc=BlockCircuit(n_qubit)
for i in range(n_qubit//3):
    bc.add_block(HardwareEfficientEntangler([3*i,3*i+1,3*i+2]))
print("Saparable sets:",bc.get_disjoint_active_sets())
n_para=bc.count_n_parameter()
start_time=time()
temp=5 # Adjustable!!
for i in range(temp):
    bc.adjust_all_parameter_by_list([0.1]*n_para)
    amp=get_0000_amplitude_on_sparse_circuit(bc)
    print(amp)
sparse_time=time()-start_time
print("Time used by using sparse circuit:",sparse_time)
bc.adjust_all_parameter_by_list([-0.1*temp]*n_para)
start_time=time()
for i in range(temp):
    bc.adjust_all_parameter_by_list([0.1]*n_para)
    amp=get_0000_amplitude_on_circuit(bc)
    print(amp)
no_sparse_time=time()-start_time
print("Time used without using sparse circuit:",no_sparse_time)
print("Sparse implementation is",no_sparse_time/sparse_time,"times faster!!")

Saparable sets: [{0, 1, 2}, {3, 4, 5}, {8, 6, 7}, {9, 10, 11}, {12, 13, 14}, {16, 17, 15}, {18, 19, 20}, {21, 22, 23}]
(0.31643648420372394-0.8286465321508206j)
(-0.47680521853099184-0.39602724229055336j)
(-0.27808908430338836+0.2002246750616874j)
(0.07072257505597296+0.1336441867269585j)
(0.04716206450560263-0.026520164562161726j)
Time used by using sparse circuit: 0.45220947265625
(0.3164364842037241-0.8286465321508201j)
(-0.4768052185309917-0.39602724229055325j)
(-0.27808908430338836+0.20022467506168745j)
(0.07072257505597296+0.1336441867269584j)
(0.047162064505602556-0.026520164562161688j)
Time used without using sparse circuit: 28.63447070121765
Sparse implementation is 63.32125360625591 times faster!!


In Mizore, the `SubspaceSolver` can be changed to sparse mode by a simply turning option `sparse_circuit` on as follows.

In [8]:
SubspaceSolver(krylov_circuits,energy_obj.hamiltonian,progress_bar=False,sparse_circuit=True)

3 states used to construct the subspace, where the complete space is 16-dimensional.


In [10]:
from Blocks import BlockCircuit
from Blocks import HardwareEfficientEntangler,RotationEntangler
# Build a block circuit
n_qubit=4
bc=BlockCircuit(n_qubit)
bc.add_block(RotationEntangler((0,1),(3,2))) #e^{i Z_0 Y_1 t}
bc.add_block(RotationEntangler((2,3),(2,1))) #e^{i Y_2 X_3 t}
bc.add_block(HardwareEfficientEntangler((2,3)))
# Print a circuit
print(bc)
# Set only 0th,1st block adjustable
bc.active_position_list=[0,1]
# Get number of gates used
print("Gate used:",bc.get_gate_used())
# Get separable qubit sets
print("Disjoint sets",bc.get_disjoint_active_sets())

Block Num:3; Qubit Num:4
Block list:
Type:RotationEntangler; Para Num:1; Qsubset:(0, 1); Pauli:ZY
Type:RotationEntangler; Para Num:1; Qsubset:(2, 3); Pauli:YX
Type:HardwareEfficientEntangler; Para Num:6; Qsubset:(2, 3)
Gate used: {'CNOT': 6, 'SingleRotation': 10, 'TimeEvolution': 0}
Disjoint sets [{0, 1}, {2, 3}]
